In [1]:
# !pip install hdfs
# ref : https://hdfscli.readthedocs.io/en/latest/quickstart.html#python-bindings

In [2]:
from hdfs import InsecureClient # 하둡 서버와 연결하는 client 객체 모듈(hdfs 접근)
import requests # web api 사용 위함
import json # json 형식으로 바꿔서 저장하기 위해 사용
import datetime as dt

In [ ]:
# 코로나 data Pipleline
1. 데이터 수집 후 dw 인 hdfs 에 저장 : 현재 note 에서 프로그래밍
2, dw에 있는 data를 가공 처리해서 dw db에 저장
3.

### DataW에 DataL 의 필요 data 를 이동 저장 단계
- Lake 에서 HDFS로 이동 저장
1. 코로나 발생현황(일간데이터) : api 수집
2. 코로나 예방접종현황(일간데이터) : web 페이지 크롤링
3. 인구 사회 데이터 : 파일 데이터
- 1, 2번 데이터는 json 형식으로 변환 후 hdfs에 저장
- 3번 데이터는 파일 그대로 저장

## HDFS handle
- port 를 명시적으로 변경하지 않았으면
- hdfs 의 node 들의 관리 서버 : localhost: 9000
- http 연결 웹 이용 서버 연결 : 'http://localhost:9870'

In [3]:
## hdfs client 연결객체 생성
## insecureclient(연결할 hdfs 서버주소: 포트, user = 사용자계정)
client = InsecureClient('http://localhost:9870', user = 'lab09')

## hdfs로 부터 읽기

In [4]:
# client 객체 사용 file 읽어오기
# hdfsclient.read(파일명) : 해당 파일에 대해 읽기 위한 연결을 생성
# 생성객체.read() - byte로 읽어온다.
with client.read('/rdd/score.txt') as reader :
    score = reader.read() # byte code 로 읽어옴

In [5]:
score

b'\xea\xb9\x80\xec\xb2\xa0\xec\x88\x98 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 50\r\n\xed\x99\x8d\xea\xb8\xb8\xeb\x8f\x99 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 80\r\n\xec\x9e\x84\xea\xba\xbd\xec\xa0\x95 \xec\x8a\xa4\xed\x8c\x8c\xed\x81\xac 60\r\n\xec\x9e\x84\xec\x9a\x94\xed\x99\x98 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 100\r\n\xed\x99\x8d\xec\xa7\x84\xed\x98\xb8 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 22\r\n\xed\x99\x8d\xec\xa7\x84\xed\x98\xb8 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 22\r\n\xec\x9d\xb4\xec\x9c\xa4\xec\x97\xb4 \xed\x85\x90\xec\x84\x9c\xed\x94\x8c\xeb\xa1\x9c\xec\x9a\xb0 90\r\n\xec\xb5\x9c\xec\x97\xb0\xec\x84\xb1 \xec\x9e\xa5\xea\xb3\xa0 100\xec\xb5\x9c\xec\x97\xb0\xec\x84\xb1 \xec\x9e\xa5\xea\xb3\xa0 100'

In [6]:
# https://docs.python.org/ko/3/library/stdtypes.html#bytes-and-bytearray-operations
score_data = bytes.decode(score)
print(score_data)

김철수 스파크 50
홍길동 스파크 80
임꺽정 스파크 60
임요환 텐서플로우 100
홍진호 텐서플로우 22
홍진호 텐서플로우 22
이윤열 텐서플로우 90
최연성 장고 100최연성 장고 100


## hdfs에 쓰기

#### hdd(note 디렉터리)에 있는 data 를 hdfs 로 이동
1. 로컬 데이터 파일을 열기
2. hdfs client 통해서 쓰기 모듈 사용
3. 로컬 파일의 내용을 한줄씩 읽어서 -> hdfs에 쓰기

In [7]:
with open('./data/corona_data/sido_area.csv',encoding='CP949') as reader, client.write('/corona_data/sido_area_tmp1.csv') as writer:
    for line in reader :
        writer.write(line.encode('CP949'))

In [9]:
# 폴더 전송 - hdfs.upload(저장할 곳, 원본)
# aws 서버에서 진행하고 있기 때문에 권한이 없으면 오류가 발생한다. 밑에서도 계속 발생할 예정임
client.upload('/corona_data/loctmp', './data/corona_data')

'/corona_data/loctmp'

## hdfs의 파일 일부 수정하기
- hdfs.client.write (파일명, append = True)

In [10]:
client.write('/rdd/score.txt', '\n최연성 장고 100'.encode('UTF-8'), append = True)

## hdfs 권한 수정
- 권한 777은 누구나 다 사용할 수 있음

In [37]:
client.set_permission('/corona_data/', '777')

## hdfs 삭제

In [39]:
client.delete('/corona_data/loc/sido_area.xlsx')

True

### 공공데이터 REST_API 통해서 데이터 수집 후 hdfs에 저장

1. 일간 코로나 발생 데이터(전국)

## REST_API로 데이터를 호출해 HDFS에 저장
- restapi 연결을 위한 객체 생성 : request 모듈 사용
    - 함수로 구성
- 일간 데이터이기 때문에 어느 시점의 데이터를 다운받을 것이지를 위한 시점 생성 함수 구성

### restapi 연결을 위한 객체 생성 및 데이터 받아오는 함수 생성
- 데이터 반환 여부 확인 후 반환되었으면 content(text)만 반환

In [11]:
def executeRestApi(method, url, headers, params):  
    # raise Exception('응답코드 : 500')  # 예외 테스트
    # err_num = 10/0 # 예외 테스트
    if method == "get":
        res = requests.get(url, params=params, headers=headers)
    else:
        res = requests.post(url, data=params, headers=headers)

    if res == None or res.status_code != 200:
        raise Exception('응답코드 : ' + str(res.status_code))
       
    return res.text

### 기준일자 함수

In [12]:
def cal_std_day(befor_day):
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

In [14]:
cal_std_day(365)

'2022-12-26'

### logger
- log 를 기록해주는 객체(관련 패키지 존재 : logging)
- 외부 연결 시 오류 상태 확인 위해서 사용자 정의 log 사용 가능해야 한다.
- restapi 는 외부 서버 연결이므로 log 기록
- 로그 관련 디렉터리 
    1. 로그 관련 패키지 import(import logging)
    2. 로그 관리 객체 logger 생성(logging.getLogger('로거이름'))
    3. 로그 파일 핸들러 생성(logging.FileHandler('로그파일명.log))
    4. 로거에 파일핸들러 추가

In [15]:
import logging

- 아래 log 만드는 코드 프로젝트나 자동화 진행시에는 함수로 생성하는게 일반적임

In [16]:
co_logger = logging.getLogger('conrona_api')
f_handler = logging.FileHandler('./log/rest_api/'+cal_std_day(0)+'log')
co_logger.addHandler(f_handler)

In [17]:
# 이렇게 문자열로 찍을 수도 있음
# json양식으로 log를 저장하면 다음에 읽어와서 처리하기도 편하다.
# 에러 발생 시 호출해주는 함수이다.
# log 파일에 error('log 문자열') log 문자열 기록
co_logger.error('코로나 발생현황 다운로드 에러 발생')
# co_logger.error('corona_patient_'+cal_std_day(0)+'.json 다운로드 실패')

- 위에서 강제 발생시킨 에러에 대한 log 는 ./log/rest_api/'+cal_std_day(0)+'log 로그 파일에서 확인

### api 호출
- 관련 서비스키, url 저장할 때 패쓰 및 파일명
     - 전역변수로 설정하거나, 함수 구성 후 리턴 받아 사용한다.

In [24]:
url = 'http://apis.data.go.kr/1352000/ODMS_COVID_04/callCovid04Api'
service_key = 'vpA5w%2Frqnqv5OVUvdS1jzdjrdM3lN%2B3Omjbnx64dXdMBMpL7xbLreYIFaigNiexQelZHSsPNMGL3A91tHKF1YQ%3D%3D'
file_dir = '/corona_data/patient/' # hdfs 저장 경로

In [20]:
# std_day : 오늘부터 얼머나 이전 데이터인지 나타냄 0 : 당일/ 1: 어제 / 365 : 1년전
def create_param(std_day):
    return {'serviceKey' : service_key,
           'pageNo' : 1,
           'numOfRows' : 500,
           'apiType' : 'JSON',
            'std_day': cal_std_day(std_day)}

In [21]:
create_param(365)

{'serviceKey': 'vpA5w%2Frqnqv5OVUvdS1jzdjrdM3lN%2B3Omjbnx64dXdMBMpL7xbLreYIFaigNiexQelZHSsPNMGL3A91tHKF1YQ%3D%3D',
 'pageNo': 1,
 'numOfRows': 500,
 'apiType': 'JSON',
 'std_day': '2022-12-26'}

### 수집 관련 코드
- 함수로 구성 필요

In [25]:
for i in range(365, 367) :
    params = create_param(i) # api 요청 파라미터 생성
    log_dict = {
        'is_sucess' : 'Fail'
        ,'type' : 'corona_patient'
        ,'std_day' : params['std_day']
        ,'params' : params
    }
    try:
        res = executeRestApi('get',url, {}, params) # 공공데이터 api에서 추출
        file_name = 'corona_patient_'+cal_std_day(i)+'.json' # 저장 파일명 생성
        client.write(file_dir+file_name,res, encoding = 'utf=8') # hdfs 에 저장
    
    except Exception as e :
        log_dict['err_msg'] = e.__str__() # exception 객체 내 에러메시지를 반환
        log_json = json.dumps(log_dict, ensure_ascii=False) # json형태로 기록 
        co_logger.error(log_json) # 로그파일에 기록

## 코로나 백신 접종 현황데이터
#### 보건 복지부 제공 웹페이지 통해서 데이터 수집 후 hdfs에 저장
2. 일간 코로나 발생데이터(전국)

## 웹크롤링 hdfs 저장

- 예방접종현황(백신 접종현황)
- json 형태로 구성 후 hdfs 에 저장

In [26]:
# pip install BeautifulSoup4
from bs4 import BeautifulSoup

In [27]:
url = 'https://ncv.kdca.go.kr/mainStatus.es?mid=a11702000000'

In [28]:
response = executeRestApi('get', url, {},{})
# response

In [29]:
soup = BeautifulSoup(response, 'html.parser') # 받은 웹페이지 파싱객체 생성

In [31]:
# 전체 행을 다 가져오기 위해서 뒤의 세부 항을 삭제해준다.
#content > div.data_table.tbl_scrl_mini > table > tbody > tr

In [32]:
trs = soup.select('#content > div.data_table.tbl_scrl_mini > table > tbody > tr')

In [34]:
len(trs)
trs

[<tr>
 <th scope="row">전국</th>
 <td class="ta_r">7,313</td>
 <td class="ta_r">3,902,792</td>
 <td class="ta_r">39.9%</td>
 </tr>,
 <tr>
 <th scope="row">서울</th>
 <td class="ta_r">1,520</td>
 <td class="ta_r">688,448</td>
 <td class="ta_r">39.7%</td>
 <!-- <td class="ta_r">0</td>
                 <td class="ta_r">0</td>
                 <td class="ta_r">39.7%</td>
                 <td class="ta_r">0</td>
                 <td class="ta_r">0</td>
                 <td class="ta_r"></td> -->
 </tr>,
 <tr>
 <th scope="row">부산</th>
 <td class="ta_r">450</td>
 <td class="ta_r">272,732</td>
 <td class="ta_r">36.4%</td>
 <!--  <td class="ta_r">0</td>
                 <td class="ta_r">0</td>
                 <td class="ta_r">36.4%</td>
                 <td class="ta_r">0</td>
                 <td class="ta_r">0</td>
                 <td class="ta_r"></td> -->
 </tr>,
 <tr>
 <th scope="row">대구</th>
 <td class="ta_r">347</td>
 <td class="ta_r">150,718</td>
 <td class="ta_r">32.1%</td>
 <!-- <td cla

In [35]:
cols = ['loc', 'new', 'accumulate', 'rate']
data = [] # 전국 데이터가 key-value dict 로 저장

In [36]:
for idx, tr in enumerate(trs):
    if idx == 0:
        continue # idx 0번 데이터는 전국 데이터 제외해야 하므로 continue
    tuple_t = []
    # 지역 값
    loc_th = tr.find('th')
    # 접종관련 데이터들
    tds = tr.select('td')
    # 지역 이름을 추가
    tuple_t.append(loc_th.text)
    # 접종관련 데이터 추가
    tuple_t.append(tds[0].text.replace(' ','').replace('\r\n', '').replace(',',''))
    tuple_t.append(tds[1].text.replace(' ','').replace('\r\n', '').replace(',',''))
    tuple_t.append(tds[2].text.replace(' ','').replace('\r\n', '').replace(',',''))
    data.append(dict(zip(cols, tuple_t)))

In [37]:
data

[{'loc': '서울', 'new': '1520', 'accumulate': '688448', 'rate': '39.7%'},
 {'loc': '부산', 'new': '450', 'accumulate': '272732', 'rate': '36.4%'},
 {'loc': '대구', 'new': '347', 'accumulate': '150718', 'rate': '32.1%'},
 {'loc': '인천', 'new': '396', 'accumulate': '198801', 'rate': '39.9%'},
 {'loc': '광주', 'new': '148', 'accumulate': '109748', 'rate': '46.6%'},
 {'loc': '대전', 'new': '143', 'accumulate': '103823', 'rate': '42.2%'},
 {'loc': '울산', 'new': '177', 'accumulate': '66139', 'rate': '37.4%'},
 {'loc': '세종', 'new': '45', 'accumulate': '18322', 'rate': '42.9%'},
 {'loc': '경기', 'new': '1512', 'accumulate': '863120', 'rate': '40.6%'},
 {'loc': '강원', 'new': '256', 'accumulate': '141741', 'rate': '38.5%'},
 {'loc': '충북', 'new': '231', 'accumulate': '142550', 'rate': '42.6%'},
 {'loc': '충남', 'new': '265', 'accumulate': '189942', 'rate': '41.5%'},
 {'loc': '전북', 'new': '292', 'accumulate': '197158', 'rate': '46.3%'},
 {'loc': '전남', 'new': '327', 'accumulate': '228404', 'rate': '48.1%'},
 {'loc'

In [38]:
# json 에 포함시킬 meta data 구성
res = {
    'meta':{
        'desc':'지역별 코로나 예방접종 인구 현황',
        'cols':{
                'loc':'지역'
                ,'new':'신규접종'
                ,'accumulate':'누적접종' 
               ,'rate' : '접종률'
                },
        'stdDay' : cal_std_day(0)
    },
    'data' : data
}
res

{'meta': {'desc': '지역별 코로나 예방접종 인구 현황',
  'cols': {'loc': '지역', 'new': '신규접종', 'accumulate': '누적접종', 'rate': '접종률'},
  'stdDay': '2023-12-26'},
 'data': [{'loc': '서울',
   'new': '1520',
   'accumulate': '688448',
   'rate': '39.7%'},
  {'loc': '부산', 'new': '450', 'accumulate': '272732', 'rate': '36.4%'},
  {'loc': '대구', 'new': '347', 'accumulate': '150718', 'rate': '32.1%'},
  {'loc': '인천', 'new': '396', 'accumulate': '198801', 'rate': '39.9%'},
  {'loc': '광주', 'new': '148', 'accumulate': '109748', 'rate': '46.6%'},
  {'loc': '대전', 'new': '143', 'accumulate': '103823', 'rate': '42.2%'},
  {'loc': '울산', 'new': '177', 'accumulate': '66139', 'rate': '37.4%'},
  {'loc': '세종', 'new': '45', 'accumulate': '18322', 'rate': '42.9%'},
  {'loc': '경기', 'new': '1512', 'accumulate': '863120', 'rate': '40.6%'},
  {'loc': '강원', 'new': '256', 'accumulate': '141741', 'rate': '38.5%'},
  {'loc': '충북', 'new': '231', 'accumulate': '142550', 'rate': '42.6%'},
  {'loc': '충남', 'new': '265', 'accumulate': '189

In [39]:
file_dir = '/corona_data/vaccine/'
file_name = 'corona_vaccine_65'+cal_std_day(1)+'.json'
client.write(file_dir+file_name, json.dumps(res, ensure_ascii = False), encoding='utf=8')

In [40]:
file_dir = '/corona_data/vaccine/'
file_name = 'corona_vaccine_65'+cal_std_day(1)+'.json'
vaccine = spark.read.json(file_dir+file_name)
vaccine.show()

+----------------------+------------------------------+
|                  data|                          meta|
+----------------------+------------------------------+
|[{688448, 서울, 152...|{{누적접종, 지역, 신규접종,...|
+----------------------+------------------------------+



### 위 크롤링된 데이터가 전 연령에 대한 백신 관련 데이터가 아니므로 다른 파일 사용함

In [41]:
import pandas as pd

In [42]:
fin_vac = pd.read_csv('./data/corona_data/코로나백신접종완료_2022_12.csv', index_col = 0)

In [43]:
# 1, 2, 3 차 접종의 인원들을 나타냄
fin_vac.head()

,LOC,V1,V2,V3
0,제주,591058,585437.0,439106.0
1,서울,8335298,8257768.0,6076429.0
2,부산,2879900,2851362.0,2140773.0
3,대구,2018604,1995783.0,1405163.0
4,광주,1261981,1250568.0,968947.0


In [46]:
tmp = fin_vac.loc[4]
tmp

LOC           광주
V1       1261981
V2     1250568.0
V3      968947.0
Name: 4, dtype: object

In [47]:
cols = ['loc', 'v1', 'v2', 'v3']
data = []

In [48]:
for idx in fin_vac.index:
    tuple_t = []
    tmp = fin_vac.loc[idx]
    tuple_t.append(str(tmp.LOC))
    tuple_t.append(str(tmp.V1))
    tuple_t.append(str(tmp.V2))
    tuple_t.append(str(tmp.V3))
    data.append(dict(zip(cols, tuple_t)))

In [49]:
data

[{'loc': '제주', 'v1': '591058', 'v2': '585437.0', 'v3': '439106.0'},
 {'loc': '서울', 'v1': '8335298', 'v2': '8257768.0', 'v3': '6076429.0'},
 {'loc': '부산', 'v1': '2879900', 'v2': '2851362.0', 'v3': '2140773.0'},
 {'loc': '대구', 'v1': '2018604', 'v2': '1995783.0', 'v3': '1405163.0'},
 {'loc': '광주', 'v1': '1261981', 'v2': '1250568.0', 'v3': '968947.0'},
 {'loc': '인천', 'v1': '2574233', 'v2': '2550081.0', 'v3': '1915266.0'},
 {'loc': '대전', 'v1': '1245473', 'v2': '1233376.0', 'v3': '909102.0'},
 {'loc': '경기', 'v1': '11837174', 'v2': '11723258.0', 'v3': '8707827.0'},
 {'loc': '세종', 'v1': '300568', 'v2': '296859.0', 'v3': '211095.0'},
 {'loc': '경남', 'v1': '2882823', 'v2': '2853755.0', 'v3': '2145652.0'},
 {'loc': '경북', 'v1': '2293225', 'v2': '2269593.0', 'v3': '1727188.0'},
 {'loc': '전남', 'v1': '1655633', 'v2': '1641348.0', 'v3': '1365245.0'},
 {'loc': '전북', 'v1': '1593950', 'v2': '1581091.0', 'v3': '1285767.0'},
 {'loc': '충남', 'v1': '1902008', 'v2': '1884584.0', 'v3': '1479736.0'},
 {'loc': '충북

In [50]:
# json 에 포함시킬 meta data 구성
res = {
    'meta':{
        'desc':'지역별 코로나 예방접종 인구 현황',
        'cols':{
                'loc':'지역'
                ,'V1':'1차 접종 누적'
                ,'V2':'2차 접종 누적' 
                ,'V3':'3차 접종 누적'
                },
        'stdDay' : cal_std_day(365)
    },
    'data' : data
}
res

{'meta': {'desc': '지역별 코로나 예방접종 인구 현황',
  'cols': {'loc': '지역', 'V1': '1차 접종 누적', 'V2': '2차 접종 누적', 'V3': '3차 접종 누적'},
  'stdDay': '2022-12-26'},
 'data': [{'loc': '제주', 'v1': '591058', 'v2': '585437.0', 'v3': '439106.0'},
  {'loc': '서울', 'v1': '8335298', 'v2': '8257768.0', 'v3': '6076429.0'},
  {'loc': '부산', 'v1': '2879900', 'v2': '2851362.0', 'v3': '2140773.0'},
  {'loc': '대구', 'v1': '2018604', 'v2': '1995783.0', 'v3': '1405163.0'},
  {'loc': '광주', 'v1': '1261981', 'v2': '1250568.0', 'v3': '968947.0'},
  {'loc': '인천', 'v1': '2574233', 'v2': '2550081.0', 'v3': '1915266.0'},
  {'loc': '대전', 'v1': '1245473', 'v2': '1233376.0', 'v3': '909102.0'},
  {'loc': '경기', 'v1': '11837174', 'v2': '11723258.0', 'v3': '8707827.0'},
  {'loc': '세종', 'v1': '300568', 'v2': '296859.0', 'v3': '211095.0'},
  {'loc': '경남', 'v1': '2882823', 'v2': '2853755.0', 'v3': '2145652.0'},
  {'loc': '경북', 'v1': '2293225', 'v2': '2269593.0', 'v3': '1727188.0'},
  {'loc': '전남', 'v1': '1655633', 'v2': '1641348.0', 'v3': '1

In [51]:
file_dir = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(365)+'.json'
client.write(file_dir+file_name, json.dumps(res, ensure_ascii = False), encoding='utf=8')

In [52]:
file_dir = '/corona_data/vaccine/'
file_name = 'corona_vaccine_'+cal_std_day(365)+'.json'
vaccine = spark.read.json(file_dir+file_name)
vaccine.show()

+----------------------+----------------------------+
|                  data|                        meta|
+----------------------+----------------------------+
|[{제주, 591058, 585...|{{1차 접종 누적, 2차 접종...|
+----------------------+----------------------------+



### WEB DATA 를 수집하고 JSON 형식으로 처리해서 HDFS 에 저장 ETL 과정을 진행함